In [1]:
import numpy as np

# ✅ Step 1: 상수 정의
rho = 1000            # 물의 밀도 [kg/m^3]
sigma = 0.0728        # 물의 표면장력 [N/m]

# 변수 정의
D_p = 5.26 * 1e-3            # 피스톤 직경 [m]
D_out2 = 500 * 1e-6     # 노즐 출구 직경 [m]
k1 = 1300             # 하부 스프링 상수 [N/m] -> 만능시험기 
k2 = 2600             # 상부 스프링 상수 [N/m]
x = 0.02              # 피스톤 압축 거리 [m]
F_finger = 18         #[N] -> 만능시험기 

D_in = 1.06 * 1e-3    # m 노즐 인렛 직경
L_in = 60 * 1e-3      # m 노즐 인렛 길이
f = 0.03              # 마찰 계수

K = 1.1               # 손실 계수 

# ✅ Step 3: 등가 스프링 상수 계산 (직렬 연결)
k_eq = (k1 * k2) / (k1 + k2)
F_total = F_finger - k_eq * x
#F_total = 0.006

# ✅ Step 2: 피스톤 단면적 계산
A_p = (np.pi * D_p**2) / 4  # [m^2]

# ✅ Step 4: 출구 속도 계산 (베르누이 기반)
m = 0.9*1e-3 #################################################################################################################################
v_chamber = m/(rho*0.2*A_p)
#v_chamber=0.176
#v_chamber=np.sqrt((2*F_total)/(rho*A_p))
#v_chamber=v_chamber-(v_chamber*0.89*A_p)/(4.15*1e-3)

v_out2 = (1/np.sqrt(1+f*(L_in/D_in)+K))*(D_p**2/D_out2**2)*np.sqrt((8*F_total) / np.pi*rho)
v_hs   = (1/np.sqrt(1+f*(L_in/D_in)+K))*(D_p**2/D_out2**2)*v_chamber
감속텀 = (1/np.sqrt(1+f*(L_in/D_in)+K))
직경텀 = D_p**2/D_out2**2

# ✅ Step 5: 웨버수 계산
We = (rho * v_out2**2 * D_out2) / sigma
We_hs = (rho * v_hs**2 * D_out2) / sigma

# ✅ Step 6: 결과 출력
print(f"식 첫째 항: {1/np.sqrt(1+f*(L_in/D_in)+K):.2f}")
print(f"식 둘째 항: {(D_p/D_out2**2):.2f}")
print(f"식 셋째 항: {np.sqrt((8*F_total) / np.pi*rho):.2f}")
print()
print(f"알짜 힘: {F_finger - k_eq * x:.2f} N")
print(f"출구 속도: {v_out2:.3f} m/s")
print(f"출구 속도 hs: {v_hs:.3f} m/s")
print(f"감속텀: {감속텀:.3f} m/s")
print(f"직경텀: {직경텀:.3f} m/s")
print(f"속도텀 제외: {감속텀*직경텀:.3f} m/s")
print(f"예상 속도텀   : {10/(감속텀*직경텀):.3f} m/s")
print(f"챔버 내부 속도: {v_chamber:.3f} m/s")
print(f"웨버수   : {We:.2f}")
print(f"웨버수 hs: {We_hs:.2f}")

식 첫째 항: 0.51
식 둘째 항: 21040.00
식 셋째 항: 41.20

알짜 힘: 0.67 N
출구 속도: 2339.763 m/s
출구 속도 hs: 11.760 m/s
감속텀: 0.513 m/s
직경텀: 110.670 m/s
속도텀 제외: 56.787 m/s
예상 속도텀   : 0.176 m/s
챔버 내부 속도: 0.207 m/s
웨버수   : 37599540.22
웨버수 hs: 949.81
